In [2]:
import time
import pandas as pd
import numpy as np
import re
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn import metrics
import copy

In [3]:
# _______________________________第一部分 生成userid_cateid_new.csv_________________________________________

# 计时
logfile1 = open("logfile_1.txt",'w')
logfile1.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' begin one part\n')

# 读入用户ID，存入user_id_list列表
candidata = open('candidate.txt')                                # txt文件，每行代表用户ID
user_id_list = [line for line in candidata]
# print(user_id_list)                                            # 28501行数据构成的超长序列
us_len = len(user_id_list)
for a in range(us_len):
    user_id_list[a] = re.compile(r'\n').sub('',user_id_list[a])  # 正则化字符串处理，应该是去掉换行符号
# print(user_id_list)

# 读入all_news_info,商品类别存入cate_id_list
all_news_info = pd.read_csv('all_news_info.csv')   # 读取所有新闻数据75213×3，新闻唯一编号，划分类别，时间戳（发布时间？）
cate_id = all_news_info.cate_id
cate_id_list = list(set(cate_id))                  # 利用集合性质去掉重复元素，构成纯划分类别
# print(all_news_info[0:10])
# print(len(cate_id_list))

# 读入train
train = pd.read_csv('train.csv')                   # 读取训练集，3272044×5，用户ID，新闻编号，划分类别，行为类型，时间戳（浏览时间）

# 构建DataFrame

def cal_eve_user(user):                            # 定义函数，计算每一个用户的？？？
    user_train = train.ix[train.user_id==user]     # 训练集的用户ID为自变量输入的所有行？？？
    user_items = list(set(user_train.item_id))     # 该用户行为行的所有浏览新闻编号，同样集合消重（可能存在反复浏览）
    def find_itemcate(item):                       # 定义内部函数，寻找新闻编号→类别
        if item not in list(all_news_info.item_id): # 对于该新闻编号，若不在所有新闻的编号总列中，则直接返回
            return
        onepart_item_cate = list(all_news_info.ix[all_news_info.item_id==item].cate_id)[0] # 对于该新闻编号的所属类别，但只取第一个（如果有重复，那么都是一样的）
        return onepart_item_cate                    # 返回该物品编号的类别
    onepart_user_cate_list = list(map(find_itemcate,user_items))# 对该用户的所有浏览新闻编号进行类别检索
    auser_cate_num_list = list(map(lambda cate:onepart_user_cate_list.count(cate),cate_id_list))# 对该用户的浏览新闻类别进行数量统计
    return auser_cate_num_list
all_user_cate = list(map(cal_eve_user,user_id_list))# 对所有用户进行上述函数操作
user_cate_df = pd.DataFrame(all_user_cate)
user_cate_df.index = user_id_list
user_cate_df.columns = cate_id_list

# print(user_cate_df)

user_cate_df.to_csv('userid_cateid_new.csv',index=False) # 数据转化为CSV，并且将index行隐藏，得到每个用户的浏览新闻类别统计矩阵（28501×38）
print("输出完成")
logfile1.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' finished one part')
logfile1.close()


['15775343\n', '14621284\n', '11022650\n', '2539552\n', '5445998\n', '10915379\n', '16083308\n', '2758315\n', '2758317\n', '16983589\n', '6505788\n', '7476754\n', '2758318\n', '10257598\n', '15004024\n', '11956044\n', '15004020\n', '8846346\n', '6839310\n', '7893483\n', '8846343\n', '16426655\n', '16426656\n', '16426657\n', '15924795\n', '16426653\n', '13111273\n', '13111270\n', '324701\n', '12180327\n', '11870754\n', '11870756\n', '16628405\n', '11329283\n', '1736494\n', '1736497\n', '16520989\n', '865644022442890\n', '15103080\n', '15103081\n', '15103082\n', '16520983\n', '3990569\n', '12814585\n', '12814584\n', '12614051\n', '12614056\n', '4087525\n', '14603755\n', '4471800\n', '3726670\n', '14576291\n', '14576293\n', '13307019\n', '13307018\n', '13307013\n', '13414152\n', '13307011\n', '7855576\n', '1394812\n', '10459242\n', '9822963\n', '10459244\n', '10459248\n', '13311468\n', '15759818\n', '13311461\n', '5619193\n', '13311465\n', '9641539\n', '15082987\n', '16612847\n', '4117606

i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


          1_8  1_2  4_1  1_14  1_3  1_9  1_6  3_20  1_10  3_3  ...  3_8  1_5  \
15775343    0    5    0     0    0    0   15     0     0    0  ...    2   18   
14621284    0    1    0     0    1    0    0     0     0    0  ...    0    0   
11022650    8    0    0     0    8    0    8     0     0    0  ...    0    0   
2539552     0    0    0     0    6    0   15     0     0    0  ...    0    0   
5445998     0    3    0     0    1    1    1     0     1    0  ...    0    0   
...       ...  ...  ...   ...  ...  ...  ...   ...   ...  ...  ...  ...  ...   
4935797     0    0    0     0    2    0    1     0     0    0  ...    0    0   
2748304     1    1    0     3    1    0    3     0     0    0  ...    0    7   
9297696     0    0    0    11    7    0   37     0     0    0  ...    0    0   
16618155    0    6    0     0    1    0   14     0     0    0  ...    1    7   
9517835     0    0    0     0   27    0    1     0     0    0  ...    0    0   

          1_24  1_12  3_1  1_1  3_13  1

In [ ]:
# ----------------------------------------第二部分 生成user_type.csv---------------------------------------------
logfile2 = open("logfile_2.txt",'w')
logfile2.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' begin two part\n')

train2 = pd.read_csv('train.csv')

candidate = open('candidate.txt','r')
user_list = [line for line in candidate]
for i in range(len(user_list)):
    user_list[i] = re.compile('\n').sub('',user_list[i])

def eve_user_time(user):
    user_train = train2.ix[train2.user_id==user]
    user_time_np_list = np.array(list(user_train.action_time))
    bool_a = user_time_np_list<1487192400
    bool_b = (user_time_np_list>=1487260800)&(user_time_np_list<1487271600)
    bool_c = (user_time_np_list>=1487347200)&(user_time_np_list<1487358000)
    bool_all = list(bool_a|bool_b|bool_c)
    n = bool_all.count(True)
    if n/len(bool_all)>0.5:
        return 1
    else:
        return 0
print('----------------开始计算user_type----------------')
user_type_list = list(map(eve_user_time,user_list))
print(user_type_list)
print(len(user_type_list))
print(len(user_list))
pd.DataFrame(user_type_list).to_csv("user_type3_0.5.csv",index=False)
logfile2.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' finished two part')
logfile2.close()


In [4]:
# 在之前的两部分，程序主要完成了两个预备工作：
# ①根据用户的3天浏览记录得到用户-新闻类别的频数矩阵
# ②根据用户浏览的时间分布特点将用户分为了2类（比例很不均匀，极少部分符合要求，应该为夜猫子类型？）
# ---------------------------------------第三部分 推荐--------------------------------------
logfile3 = open("logfile_3.txt",'w')
logfile3.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' begin three part\n')

train = pd.read_csv('train.csv')                         # 读取训练数据集，3272044×5，用户ID，新闻编号，划分类别，行为类型，时间戳（浏览时间）
train.item_id = train['item_id'].astype(str)             # 训练集出现的新闻编号，转化为字符串
train.user_id = train['user_id'].astype(str)             # 训练集出现的用户id，转化为字符串

news_info = pd.read_csv('news_info.csv')                 # 待推荐的新闻数据，41252×3，新闻编号，类别和时间戳
BX_item_set = list(news_info.item_id)                    # 待推荐的新闻编号

all_news_info = pd.read_csv('all_news_info.csv')         # 所有的新闻数据，75213×3，新闻编号，类别和时间戳

candidate = open('candidate.txt','r')
user_list = [line for line in candidate]                # 所有用户ID
for i in range(len(user_list)):
    user_list[i] = re.compile('\n').sub('',user_list[i])
print(user_list)

user_cate_df = pd.read_csv('userid_cateid_new.csv')     # 载入用户-新闻类别矩阵
cateid_list = list(user_cate_df.columns)                # 导入该矩阵的列单位，即所有新闻类别

user_type = pd.read_csv("user_type3_0.5.csv")           # 载入用户类型数列
user_type_list = list(user_type['0'])                   # 取第一列化为list
print(user_type_list)

test = open('test.txt','r')                             # 打开测试集
recording = [line for line in test]                    # test的每一行作为字符串构成list
user_in_test = []
useritem_list_in_test = []                              # 其中子列表为test中每个用户的记录
for i in range(len(recording)):
    uid = re.compile(',').split(recording[i])[0]        # 每一行逗号分隔两部分的第一部分——即用户id
    user_in_test.append(uid)                            # 在测试集中的用户列表添加上这个id
    item_str = re.compile(',').split(recording[i])[1]   # 每一行逗号分隔两部分的第二部分——即用户浏览的新闻编号
    item_str = re.compile('\n').sub('',item_str)        # 去掉'\n'号，只有最后才有这个
    user_items = re.compile(' ').split(item_str)        # 按照空格划分，形成真正的浏览新闻编号list
    useritem_list_in_test.append(user_items)            # 在测试集中的用户浏览列表添加这部分编号，以list的形式添加，与用户对应
print(user_in_test)                                     # 输出所有测试集中的用户id
print(useritem_list_in_test)                            # 输出所有测试集中的用户浏览新闻的编号

# 将test数据整合成userid_cateid
test_user_cate_df = pd.DataFrame(index=user_in_test,columns=cateid_list) # 行列构框，测试集下的用户-新闻类别频数矩阵
for key,values in test_user_cate_df.items():
    _ = values.fillna(0,inplace=True)
for i in range(len(user_in_test)):
    for item in useritem_list_in_test[i]:              # 对该用户的浏览新闻编号进行判断
        if int(item) in BX_item_set:                   # 如果该新闻编号在待推荐的新闻编号中
            cate = list(all_news_info[all_news_info.item_id==int(item)].cate_id)[0] # 在全部新闻数据中检索该新闻的类别（为啥不在待推荐的新闻中检索？）
            if cate in cateid_list:                    # 如果这个类别在训练集中的新闻类别中（这个可能是新增的类别，则不计数）
                test_user_cate_df[cate][user_in_test[i]] += 1 # 则测试集下的对应的用户ID，新闻类别坐标下的频数+1


# 训练数据聚类
k = 3
km = KMeans(n_clusters=k)                         # k均值聚类，聚类数目3（x坐标有特征含义吗？）（应该是对用户进行的浏览特征进行分类）
K = km.fit((user_cate_df/3).values)               # 用户-新闻类别频数矩阵值除以3，用于求解(除以3除了缩短距离，有本质作用吗？）
train_label = km.labels_                          # 得到训练标签

# 分类器用于test数据
test_label = km.predict(test_user_cate_df.values)
print(train_label)        # 训练集的聚类标签
print(test_label)        # test聚类标签


# 将train中最后一小时数据与test放在一起
# 将test中所有item按类分开
test_item_cluster = [[],[],[]]
count_list_cluster = []
test_item_set_cluster = []
for i in range(len(user_in_test)):
    test_item_cluster[test_label[i]].extend(useritem_list_in_test[i])  # extend与append区别
for i in range(k):
    item_set = list(set(test_item_cluster[i]))  # 浏览新闻编号去重
    count_list = list(map(lambda item:test_item_cluster[i].count(item),item_set)) # 浏览新闻编号计数
    count_list_cluster.append(count_list)   # 以list形式连接，共k组，每组为相应类别簇的不重复新闻计数
    test_item_set_cluster.append(item_set)  # 同样list形式连接，共k组，每组为相应类别簇的不重复新闻编号，与上面对应
print("----------------test按聚类整理后item-------------\n",test_item_set_cluster) # 编号k组
print("----------------test按类整理后item次数---------------\n",count_list_cluster)# 计数k组

# train中最后一小时数据按类分开,与test合并
train_last_one = train[train.action_time>1487431800]
train_last_one_user = list(set(train_last_one.user_id))
train_lastone_item_cluster = copy.deepcopy(test_item_cluster)  # 先深度复制测试集下的聚类统计新闻编号
for user in train_last_one_user:
    lab = train_label[user_list.index(user)] # 这个user_id对应的用户浏览记录聚类结果
    train_user_last_one = train_last_one[train_last_one.user_id==user] # 该用户对应的训练数据行？×5
    train_user_lastone_item = list(train_user_last_one.item_id)        # 该用户对应的浏览新闻list
    train_lastone_item_cluster[lab].extend(train_user_lastone_item)    # 聚类簇新闻编号中添加这一部分
train_lastone_item_set_cluster = []
train_lastone_item_count_list = []
for i in range(k):
    item_set = list(set(train_lastone_item_cluster[i]))        # 每一个聚类簇消重
    train_lastone_item_set_cluster.append(item_set)            # 消重后作为list在套进去
    count_list = list(map(lambda item:train_lastone_item_cluster[i].count(item),item_set))
    train_lastone_item_count_list.append(count_list)           # 重新计数，再来一次

# 所以上部分的最终命名变量为train_lastone_item_set_cluster与train_lastone_item_count_list，已经包含了测试集，holyshit


def user_similar(user):      # user为user_list中的用户
    num = 100
    almost_item_list = list(set(train.ix[train.user_id==user].item_id))
    almost_cate_list = list(set(train.ix[train.user_id==user].cate_id)) # 当前用户的浏览新闻编号与类别

    if user_type_list[user_list.index(user)]==1:                      # 用聚类结果（夜猫子类？）
        lab = train_label[user_list.index(user)]                      # 用户标签簇
        user_BX_score_list = train_lastone_item_count_list[lab].copy()# 簇下的编号与计数（最后一小时的混合数据只在这里起到作用）
        user_BX_item_list = train_lastone_item_set_cluster[lab]
        print(1)

    # 之前聚类的结果居然只有在这700多个用户上起作用吗？？？
    
    else:             # 协同过滤
        # user在userid_cateid中的数据
        user_data = np.array(user_cate_df.ix[user_list.index(user)])/3 # 该用户的浏览新闻频数记录数组除以3，与之前一致

        def distance(other_row):
            similar = pearsonr(user_data,other_row)[0] # 这是什么fucker函数？？？皮尔森相关系数，去中心化的向量余弦夹角
            return similar
        user_pear_list = list(map(distance,test_user_cate_df.values)) # 测试集上的每一行进行距离计算
        user_dist_list = list(0.5+0.5*np.array(user_pear_list))       # 进行了偏置与缩放（这个值最终到了0~1）
        # 负相关被认为是某种距离0，可以理解为两个人的喜好完全相反，那么在推荐上就会完全不同
        user_BX_item_series = pd.Series()
        weight_list = []
        for i in range(num):
            max_dist = max(user_dist_list)                            # 最远的用户距离（但是是最相似的？？？？）
            max_similar_test_user_items = useritem_list_in_test[user_dist_list.index(max_dist)] # 最相似的测试用户浏览新闻编号
            user_dist_list[user_dist_list.index(max_dist)] = 0        # 把这个距离置为0
            weight = max_dist                     # 把这个距离设置为这个用户的权重
            weight_list.append(max_dist)          # 相似度前num的权重列表

            test_user_series = pd.Series(np.ones(len(max_similar_test_user_items)),index=max_similar_test_user_items) # 字典 编号→1
            a = pd.concat([user_BX_item_series,test_user_series],axis=1) # 按列拼接，初始时就是后者，即行对齐
            a = a.fillna(0)                        # 用0填充缺失值，错位就会产生NaN
            user_BX_item_series = a[0]+a[1]*weight # 新的BX列为a的前两列权重和，然后放在新的test列前面
        # print(user_BX_item_series)
        # 循环完成后得到user_BX_item_series为该用户的备选推荐集
        user_BX_item_list = list(user_BX_item_series.index) # 迭代后的BX列的编号(应该是前100接近的测试用户浏览新闻编号的不重复行)
        user_BX_score_list = list(user_BX_item_series.values)# 与编号对应的某种权重和，越大越好？
        print(0)

    # 推荐
    times = 0
    s = ''
    while times<5:
        if max(user_BX_score_list)==0:
            break
        max_score = max(user_BX_score_list)                                # 先取最大分数值
        max_item = user_BX_item_list[user_BX_score_list.index(max_score)]  # 取最大分数值对应的新闻编号
        user_BX_score_list[user_BX_score_list.index(max_score)] = 0        # 把这个最大值设置为0
        if max_item not in almost_item_list:                              # 如果这个最大分数的新闻编号不在当前用户的训练集中的浏览编号中
            if int(max_item) in BX_item_set:                               # 如果这个最大分数的新闻编号在待推荐新闻编号中
                item_cate = list(news_info[news_info.item_id==int(max_item)].cate_id)[0]  # 那么它的类别可以在待推荐新闻中检索到
                if item_cate in almost_cate_list:                          # 如果它的类别在当前用户的训练集中的浏览类别中
                    times += 1                                             # 不在训练集中的新类别如何评价吸引力？
                    s += ' '+max_item                                      # 推荐这个新闻
    result = user+ ',' + s[1:] + '\n'
    print(result)
    return result

# 从以上定义函数可以看出，推荐的新闻在满足最大分数的基础上，还需要满足这个新闻不在训练中出现而要出现在待推荐新闻中
all_result = list(map(user_similar,user_list))
file = open("similar_result_from_test_3_0.5_train0.5_100.txt",'w',encoding='utf-8')
for result in all_result:
    file.write(result)
file.close()
print("--------------finished--------------")

logfile3.write(time.strftime('%Y-%m-%d %X',time.localtime(time.time()))+' finished three part')
logfile3.close()

['15775343', '14621284', '11022650', '2539552', '5445998', '10915379', '16083308', '2758315', '2758317', '16983589', '6505788', '7476754', '2758318', '10257598', '15004024', '11956044', '15004020', '8846346', '6839310', '7893483', '8846343', '16426655', '16426656', '16426657', '15924795', '16426653', '13111273', '13111270', '324701', '12180327', '11870754', '11870756', '16628405', '11329283', '1736494', '1736497', '16520989', '865644022442890', '15103080', '15103081', '15103082', '16520983', '3990569', '12814585', '12814584', '12614051', '12614056', '4087525', '14603755', '4471800', '3726670', '14576291', '14576293', '13307019', '13307018', '13307013', '13414152', '13307011', '7855576', '1394812', '10459242', '9822963', '10459244', '10459248', '13311468', '15759818', '13311461', '5619193', '13311465', '9641539', '15082987', '16612847', '4117606', '14544200', '9535474', '5884076', '4117601', '14224836', '16103912', '16976864', '12402005', '15122859', '11777416', '16976860', '16976863', 

[0 2 2 ... 1 0 0]
[2 2 2 ... 1 2 1]
----------------test按聚类整理后item-------------
 [['480746', '463299', '547875', '558734', '563223', '522217', '558513', '256356', '561954', '563798', '555524', '557949', '558231', '561048', '549981', '480754', '558041', '555388', '553431', '557573', '566316', '558832', '491467', '554509', '559874', '527653', '561130', '555805', '492469', '561895', '367608', '564356', '558777', '521914', '565999', '547850', '478635', '551202', '554060', '531820', '555193', '479989', '475692', '565044', '567674', '456550', '519933', '564686', '543650', '460576', '559916', '548074', '564411', '542427', '476904', '566040', '557429', '556672', '525345', '557345', '518354', '512005', '495726', '549427', '554351', '552745', '558488', '527126', '485410', '559279', '473194', '462393', '548012', '558239', '558646', '565559', '558271', '562188', '565175', '558162', '537502', '522092', '551800', '522214', '558867', '542479', '496380', '542255', '485344', '552440', '463930', '560894

i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:102: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:113: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
i:\机器人小组专用\深度学习处理图像\python\lib\site-packages\ipykernel_launcher.py:131: FutureWarning: S

0
15775343,557579 558077 558871 558910 558768

0
14621284,558910 558871 558768 558834 558773

0
11022650,558788 557579 556664 552335 556700

0
2539552,557579 557018 558082 557167 557193

0
5445998,558910 558871 558768 558834 558773

0
10915379,557579 558788 555820 552335 552472

0
16083308,552472 557579 558788 558082 558910

0
2758315,557579 558788 555820 552472 552335

0
2758317,557579 558788 552472 558082 558498

0
16983589,557579 558788 558910 555820 557167

0
6505788,552472 557579 558123 558633 558788

0
7476754,558910 558768 558871 557579 558082

0
2758318,558082 472094 558498 557965 473787

0
10257598,557579 558788 558082 552472 558259

0
15004024,557579 558788 558331 558578 557167

0
11956044,558910 558871 558768 558834 558773

0
15004020,557579 557167 558331 558082 555820

1
8846346,556700 558910 552472 556664 542524

0
6839310,552472 557579 558788 558123 558633

0
7893483,552472 558910 557579 556700 558788

0
8846343,558082 472094 557579 558498 558788

0
16426655,558910 558871

0
13023297,557579 558910 558788 558871 558768

0
16973544,557579 555820 558788 557018 557167

0
13023299,557579 558082 558788 557018 552335

0
4786498,558082 472094 558498 557965 473787

0
16836581,557579 558788 555820 558266 557018

0
16836580,558331 558253 556489 556918 547935

0
16836583,558910 557579 558788 556700 558768

1
6826603,556700 557579 558910 558871 558768

0
16836584,558910 558871 556700 557579 558788

0
1911905,557579 558788 557018 558266 557193

0
3053696,558910 558871 557579 558768 556700

0
12540722,557579 558788 557018 557167 555820

1
2850115,557579 556700 558788 557167 558910

0
15372177,557579 558082 557167 471314 558331

0
11460441,558910 557579 550143 558871 556700

0
11460442,557579 558788 558910 555820 558871

0
11460443,558788 557579 558910 558871 556700

0
5423669,557579 558788 552335 557018 555820

0
11460447,552472 557579 558082 558123 558910

0
5495404,557579 552472 558788 558082 558768

0
16991287,558910 558871 558768 558834 558773

0
13393908,558788 55

...

0
5129226,557579 558331 552472 558788 556918

0
7010338,557579 558788 558082 558910 552472

0
8810439,557579 557018 555820 558266 558788

0
6510970,558077 558910 558768 552472 557579

0
3617789,557579 558788 558331 558082 552472

0
13426708,558082 472094 558498 557965 558788

0
9779877,558331 557579 558788 556816 556664

0
14512917,557579 558788 555820 552335 558625

0
15425959,558082 472094 558788 557579 558331

0
14512912,558331 558082 558788 547419 557579

0
14940389,557579 558788 555820 558625 552335

0
14940388,558082 557579 472094 557965 558498

0
13596416,552472 558910 558123 551726 558274

0
15646004,558871 557579 558910 558768 556700

0
2976584,558331 558788 558082 557579 547419

0
14325285,552472 558123 555094 558633 469605

0
10653887,558331 558788 557579 558578 547419

0
16939077,558498 558082 472094 557835 465967

0
16939076,557579 558788 557167 557018 555820

0
12237294,557579 558788 558910 556700 558871

0
13753141,557579 558788 556700 558910 558871

0
3747590,558788 557

0
12204012,558082 472094 558498 557965 473787

0
580954,557579 556664 558082 557167 558893

0
16730558,557579 558788 472094 558331 558259

0
12823244,558077 558721 558699 558768 558096

0
1439775,472094 557579 558082 558788 558498

0
12758595,557579 558788 552335 558331 558077

0
13503480,556700 558331 558910 558768 554513

0
5393916,557579 557167 552335 558788 555820

1
4625188,557579 556700 558082 558788 557167

0
4246026,558082 472094 556700 558910 473787

0
8369877,556918 556489 558331 558253 557579

0
14615134,558077 558768 558788 556664 558910

0
A00000566EDFF7,558331 558253 556700 554513 547419

0
11376503,558082 472094 558910 556700 552472

0
9233516,557579 558788 557018 555820 557167

0
5599130,558788 558331 557579 558082 556664

0
11234571,556664 552472 558788 556489 558123

0
7932113,558082 557965 473787 558498 557835

0
15396406,558077 558721 558699 558498 556664

0
15837951,557579 558788 552335 555820 558578

0
14159100,547419 558331 558253 556918 556816

0
99000694533063,

0
4116030,557579 558331 556700 542524 558788

0
11120657,558331 547419 558253 556489 556816

0
12160016,558910 558871 558768 558834 558773

0
7625114,558331 547419 558253 556595 556918

0
12080644,556918 556664 558331 558253 556816

0
5397087,558871 558910 557579 556700 558773

0
2203066,557579 558096 557018 555820 558259

0
16442777,557579 558788 557018 558266 555820

0
14127826,558910 558768 558871 558834 558773

0
16521417,552472 557579 558123 557167 558788

0
10864664,557579 558788 558768 558910 556700

0
9719057,558082 472094 558788 558498 557579

0
5665049,547419 558331 558253 556489 557167

0
6158966,558082 472094 558910 557579 556700

0
6375708,557579 558788 552472 558082 558910

0
9132794,558788 557579 558625 552335 558578

0
1770658,557579 555820 558788 552472 558625

0
14983281,558082 558910 556700 558871 472094

0
13877070,558077 558721 558498 558331 558082

0
12142755,558082 472094 557328 473787 557835

0
8916924,558910 558871 557579 558768 558834

0
9949096,558910 558871 

0
7864022,558910 558871 558768 558834 558773

0
3687205,547419 558331 556664 556489 558253

0
6668748,558082 547419 558331 556918 556816

0
5760055,557579 558788 558331 558578 556664

0
14906772,557579 558788 552335 555820 558625

0
16945360,558910 558768 558871 558834 542797

0
15348199,558910 558871 558768 558834 558773

0
16090611,557579 558910 556700 558871 558788

0
9463566,547419 558331 556816 556489 556918

0
8546395,552472 558123 555094 558633 469605

0
9466031,532955 553492 555644 532133 529781

0
5281398,558082 557579 558498 557965 557167

0
15190660,558077 558721 557579 558699 554891

0
935416,558082 472094 558498 557965 558788

0
8167725,557579 558788 555820 558910 556700

0
5958194,557579 558910 552472 558871 556700

0
8462794,558096 557579 557018 557193 555820

0
401296,558910 557579 558871 556700 558788

0
9039717,557579 558082 557167 557018 555820

0
13990473,558788 558331 557579 556700 558625

0
16001346,558910 558871 558834 558768 558773

0
5326126,557579 558788 55701

0
16564288,558082 472094 557579 558498 557965

0
4097657,557579 558788 558578 556700 558331

0
5237362,558082 558498 557965 474366 557835

0
9814748,554247 563759 547752 555415 542995

0
9153860,547419 558331 556816 556489 558082

0
10368130,558082 558498 557965 557835 472094

0
11543532,557579 558096 557167 556489 558788

0
11226800,558331 557579 558788 558082 557167

0
15944966,558082 472094 557965 558498 558910

0
A100004DECC2FD,552472 558910 556700 551726 558274

0
13767089,557579 558788 555820 557018 558259

0
14141562,558082 557965 558498 472094 557835

0
12260902,556918 547419 558331 558253 556489

1
13726456,557579 556700 558788 557167 558910

0
8888405,558082 557965 558498 557835 558788

0
16580754,556489 557579 558331 558096 556918

0
16179000,558331 556918 555258 547419 556664

0
5280031,558788 557579 555820 552335 557018

0
4935797,558910 558082 556700 558871 558768

0
2748304,558077 557579 558721 558699 552472

0
9297696,557579 557018 558788 555820 557193

1
16618155,55670